In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langgraph.graph import END, START, StateGraph, MessagesState
#from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from typing import Literal
from langgraph.graph import StateGraph, MessagesState, state
import os

In [8]:
from monitoring_module import (
    fetch_slow_queries_bigquery,
    generate_email_content,
    send_email
)

In [4]:
graph = StateGraph(dict)

In [ ]:
@tool
def fetch_queries_tool(admin_email: str) -> dict:
    """slow running query"""
    slow_queries = fetch_slow_queries_bigquery()
    return {
        "admin_email": admin_email,
        "slow_queries": slow_queries
    }

@tool
def format_queries_tool(admin_email: str, slow_queries: list) -> dict:
    """Format queries into email."""
    formatted_email = generate_email_content(slow_queries)
    return {
        "admin_email": admin_email,
        "slow_queries": slow_queries,
        "formatted_email": formatted_email
    }

@tool
def send_email_tool(admin_email: str, formatted_email: str) -> dict:
    """Send email to admin."""
    send_email(admin_email, formatted_email)
    return {
        "admin_email": admin_email,
        "formatted_email": formatted_email
    }

# Build the graph
graph = StateGraph(dict)
graph.add_node("FetchQueries", fetch_queries_tool)
graph.add_node("FormatQueries", format_queries_tool)
graph.add_node("SendEmail", send_email_tool)

graph.set_entry_point("FetchQueries")
graph.add_edge("FetchQueries", "FormatQueries")
graph.add_edge("FormatQueries", "SendEmail")
graph.set_finish_point("SendEmail")

# Compile the graph
workflow = graph.compile()


In [28]:
llm = ChatGroq(
    temperature=0, 
    model_name="llama-3.3-70b-versatile"
)

In [10]:
workflow.invoke({
    "admin_email": "deepakk073@gmail.com"
})

Email sent successfully. Response status: 202


{'admin_email': 'deepakk073@gmail.com',
 'formatted_email': "🚨 Slow Queries Detected in BigQuery\n\n\n\nUser: deepakk073@gmail.com\nJob ID: scheduled_query_682af2c9-0000-22ea-9df8-24058872df70\nSlot Time: 5481 ms\nURL: https://console.cloud.google.com/bigquery?j=your_project:region:scheduled_query_682af2c9-0000-22ea-9df8-24058872df70\nAbort: bq cancel -j scheduled_query_682af2c9-0000-22ea-9df8-24058872df70\nQuery:\n\nDECLARE getdate DATE DEFAULT CURRENT_DATE();\nDECLARE weekday STRING;\nSET weekday = FORMAT_DATE('%A', getdate);\nINSERT INTO `mlconsole-poc.test.sql_job` (getdate, weekday) VALUES (getdate, weekday);\n---\n"}

In [4]:
# Usage example
send_email(
    to_email="deepakk073@gmail.com",
    message_body="This is a test email sent using SendGrid."
)

Email sent successfully. Response status: 202


In [33]:
from langchain_core.tools import tool
from langgraph.graph import StateGraph

# Dummy helper functions
def fetch_slow_queries_bigquery():
    return ["SELECT * FROM huge_table WHERE ...", "SELECT COUNT(*) ..."]

def generate_email_content(queries):
    return "\n".join(queries)

def send_email(to, content):
    print(f"Sending email to {to}:\n{content}")

# Tools
@tool
def fetch_queries_tool(admin_email: str) -> dict:
    """Fetch slow-running queries."""
    slow_queries = fetch_slow_queries_bigquery()
    return {
        "admin_email": admin_email,
        "slow_queries": slow_queries
    }

@tool
def format_queries_tool(admin_email: str, slow_queries: list) -> dict:
    """Format queries into email."""
    formatted_email = generate_email_content(slow_queries)
    return {
        "admin_email": admin_email,
        "slow_queries": slow_queries,
        "formatted_email": formatted_email
    }

@tool
def send_email_tool(admin_email: str, formatted_email: str) -> dict:
    """Send email to admin."""
    send_email(admin_email, formatted_email)
    return {
        "admin_email": admin_email,
        "formatted_email": formatted_email
    }

# Build the graph
graph = StateGraph(dict)
graph.add_node("FetchQueries", fetch_queries_tool)
graph.add_node("FormatQueries", format_queries_tool)
graph.add_node("SendEmail", send_email_tool)

graph.set_entry_point("FetchQueries")
graph.add_edge("FetchQueries", "FormatQueries")
graph.add_edge("FormatQueries", "SendEmail")
graph.set_finish_point("SendEmail")

# Compile the graph
workflow = graph.compile()

# ✅ Invoke with only the required input to the first node
workflow.invoke({
    "admin_email": "deepakk073@gmail.com"
})


Sending email to deepakk073@gmail.com:
SELECT * FROM huge_table WHERE ...
SELECT COUNT(*) ...


{'admin_email': 'deepakk073@gmail.com',
 'formatted_email': 'SELECT * FROM huge_table WHERE ...\nSELECT COUNT(*) ...'}

In [17]:
from google.cloud import bigquery
#client = bigquery.Client(credentials=credentials, project="mlconsole-poc")
client = bigquery.Client()
query = """
SELECT job_id,'deepakk073@gmail.com' user_email, query, total_slot_ms, creation_time 
FROM `region-us`.INFORMATION_SCHEMA.JOBS_BY_PROJECT 
WHERE 
    state = 'DONE' 
    AND job_type = 'QUERY' 
    AND creation_time > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 10 HOUR) 
    AND total_slot_ms > 5000
    limit 1
"""
job = client.query(query, job_config=bigquery.QueryJobConfig(
))
print (job)

QueryJob<project=mlconsole-poc, location=US, id=5d2d389f-87ef-4eb2-bcf1-0f0b234150b5>


In [55]:
!pip install --upgrade google-cloud google-cloud-bigquery

In [ ]:
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."
    python -m ipykernel install --user --name .venv --display-name "Aiagent-langgraph"

In [8]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END